In [1]:
import pandas as pd
from datasets import Dataset

df = pd.read_excel("dataset\MeIA_2025_train.xlsx")
df["labels"] = df["Polarity"]-1

df = df[["Review", "labels"]]
df["labels"] = df["labels"].astype(int)

df

<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Saman\AppData\Local\Temp\ipykernel_11468\2798356087.py:4: SyntaxWarning: invalid escape sequence '\M'
  df = pd.read_excel("dataset\MeIA_2025_train.xlsx")


,Review,labels
0,Un Restaurante te invita por su ambiente tan a...,1
1,Pagamos 25 pesos por la entrada y no es gran c...,2
2,Mi esposa y yo nos alojamos en el Dreams por 4...,2
3,"La única decepción puede no ser José Cuervo, p...",1
4,Cuando leí los comentarios sobre cómo son las ...,0
...,...,...
4995,"Olas decentes, infinidad de cielo azul y vista...",4
4996,El Hotel no cumplió con la reservación que hic...,0
4997,Gran lugar para cualquiera que esté interesado...,3
4998,el ambiente del restaurante muy relajado la co...,3


In [2]:
dataset = Dataset.from_pandas(df)

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(batch["Review"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [4]:
dataset = dataset.train_test_split(test_size=0.3)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [5]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=5
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
!pip install transformers==4.52.3 --force-reinstall



  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
Using cached transformers-4.52.3-py3-none-any.whl (10.5 MB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------------------------------------- - 2.4/2.4 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.7 MB 11.8 MB/s eta 0:00:01
   --------------- ------------------------ 5.0/12.7 MB 11.7 MB/s eta 0:00:01
   ----------------------- ---------------- 7.3/12.7 MB 11.7 MB/s eta 0:00:01
   ------------------------------ --------- 9.7/12.7 MB 11.6 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.7 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 12.7/12.7 MB 11.0 MB/s eta 0:00:00
Using cached packaging-25.0-py3-none-any.wh

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.


In [9]:
!pip install "fsspec<=2025.3.0"


  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./sentiment_model",
    eval_strategy="epoch",    
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

trainer.train()


C:\Users\Saman\AppData\Local\Temp\ipykernel_11468\3357229639.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Saman\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
from sklearn.metrics import classification_report

preds_output = trainer.predict(eval_dataset)
preds = preds_output.predictions.argmax(axis=1)
labels = preds_output.label_ids

print(classification_report(labels, preds, digits=3))


In [ ]:
test_df = pd.read_excel("test.xlsx")

test_ds = Dataset.from_pandas(test_df)
test_ds_tokenized = test_ds.map(tokenize, batched=True)

test_preds = trainer.predict(test_ds_tokenized)
test_labels = test_preds.predictions.argmax(axis=1)

test_df["Predicted_Polarity"] = test_labels + 1
test_df.to_excel("test_con_predicciones.xlsx", index=False)
